In [40]:
import sys
import pandas as pd
import pandas as pd
from pandas import DataFrame
from pandas import concat
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

### EXTRACT CSV INTO DATAFRAMES

In [41]:
# File to load
file_to_load = "Amazon_rev_modified.csv"

In [42]:
# Put into DataFrame
data = pd.read_csv(file_to_load)

In [43]:
# Print Data Frame
data.head()

,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Lowercase Text Clean,Positive Count,Negative Count
0,0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,i have bought several of the vitality canned d...,0,0
1,1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,product arrived labeled as jumbo salted peanut...,1,0
2,2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,this is a confection that has been around a fe...,0,0
3,3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,if you are looking for the secret ingredient i...,0,0
4,4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...,great taffy at a great price there was a wide...,2,0


In [44]:
# Print info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Unnamed: 0              100 non-null    int64 
 1   Id                      100 non-null    int64 
 2   ProductId               100 non-null    object
 3   UserId                  100 non-null    object
 4   ProfileName             100 non-null    object
 5   HelpfulnessNumerator    100 non-null    int64 
 6   HelpfulnessDenominator  100 non-null    int64 
 7   Score                   100 non-null    object
 8   Time                    100 non-null    int64 
 9   Summary                 100 non-null    object
 10  Text                    100 non-null    object
 11  Lowercase Text Clean    100 non-null    object
 12  Positive Count          100 non-null    int64 
 13  Negative Count          100 non-null    int64 
dtypes: int64(7), object(7)
memory usage: 11.1+ KB


### TRANSFORM DATA FRAME

In [46]:
# Rename DF & Select Columns to work with
data_df = data[['Id','ProductId','UserId','ProfileName','Score','Time', 'Summary','Text','Lowercase Text Clean', 'Positive Count', 'Negative Count' ]]


In [47]:
# Rename Columns
data_df.columns = ['review_id', 'product_id', 'user_id', 'profile_name', 'score', 'time', 'summary', 'text', 'lower_case_clean', 'positive_count', 'negative_count']

In [48]:
# Print DF
data_df.head(3)

,review_id,product_id,user_id,profile_name,score,time,summary,text,lower_case_clean,positive_count,negative_count
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,i have bought several of the vitality canned d...,0,0
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,product arrived labeled as jumbo salted peanut...,1,0
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,this is a confection that has been around a fe...,0,0


In [49]:
# View lenght of data
len(data_df)

100

In [50]:
# Compare vs unique review_id
len(data_df.review_id.unique())

100

In [51]:
datatime_df = data_df[['score','time']]
datatime_df

,score,time
0,positive,1303862400
1,negative,1346976000
2,positive,1219017600
3,negative,1307923200
4,positive,1350777600
...,...,...
95,positive,1326412800
96,positive,1308700800
97,positive,1295308800
98,positive,1243728000


In [52]:
# TRANSFORM COLUMN "TIME" FROM UNIX TO DATETIME
converted_df = pd.to_datetime(datatime_df['time'], unit='s')
converted_df

0    2011-04-27
1    2012-09-07
2    2008-08-18
3    2011-06-13
4    2012-10-21
        ...    
95   2012-01-13
96   2011-06-22
97   2011-01-18
98   2009-05-31
99   2011-09-19
Name: time, Length: 100, dtype: datetime64[ns]

In [53]:
# Count Positive & Negative
count = data_df['score'].value_counts()
count.head

<bound method NDFrame.head of positive    88
negative    12
Name: score, dtype: int64>

### CREATE DATABASE CONNECTION

In [61]:
connection_string = "postgres:B00tc@mp@localhost:5432/Amazon"

In [62]:
# confirm tables
engine = create_engine(f"postgresql+psycopg2://{connection_string}")

### LOAD DATAFRAME INTO DATABASE

In [63]:
data_df.to_sql(name="data", con=engine, if_exists="append", index=False)

In [68]:
# Create DF for table 'user'
user_df_table = data_df[['user_id','profile']]

In [69]:
# Identify Duplicated
user_df_table = user_df_table.assign(repeat=lambda r:r['user_id'].duplicated())

In [71]:
# Query
user_df_table = user_df_table.query('repeat == False')

In [73]:
# Drop repeated
user_df_table = user_df_table.drop(columns=['repeat'])

In [75]:
# Send data to SQL
user_df_table.to_sql(name='user',  con=engine, if_exists="append", index=False)